# Calculate normalised enrichment scores of Danaher gene sets

Script related to figure 3b

### Aim:
- Calculate TCGA NES of Danaher Gene Sets

In [1]:
sessionInfo()

R version 3.5.0 (2018-04-23)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS High Sierra 10.13.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.5.0  IRdisplay_0.6.1 pbdZMQ_0.3-3    tools_3.5.0    
 [5] htmltools_0.3.6 base64enc_0.1-3 crayon_1.3.4    Rcpp_1.0.1     
 [9] uuid_0.1-2      IRkernel_0.8.14 jsonlite_1.6    digest_0.6.18  
[13] repr_0.17       evaluate_0.13  

## Load packages

In [1]:
library(GSVA)
library(BBmisc)
library(nortest)

Warning message:
“package ‘GSVA’ was built under R version 3.5.1”
Attaching package: ‘BBmisc’

The following object is masked from ‘package:base’:

    isFALSE



In [2]:
setwd("~/git_repos/HGSOC_TME_Heterogeneity/Scripts/3/")

## Load expression data

In [ ]:
mix <- "../../Data/1/TCGAov_RNAseq.txt"

In [3]:
data <- read.csv(mix,
                 sep='\t',
                 header=T,
                 row.names='GeneSymbol')

In [4]:
head(data)

,TCGA.04.1348.01,TCGA.04.1357.01,TCGA.04.1362.01,TCGA.04.1364.01,TCGA.04.1365.01,TCGA.04.1514.01,TCGA.04.1519.01,TCGA.09.0364.01,TCGA.09.0366.01,TCGA.09.0367.01,⋯,TCGA.61.2102.01,TCGA.61.2104.01,TCGA.61.2109.01,TCGA.61.2110.01,TCGA.61.2111.01,TCGA.61.2113.01,TCGA.OY.A56P.01,TCGA.OY.A56Q.01,TCGA.VG.A8LO.01,TCGA.WR.A838.01
A1BG,66.4695,65.5664,41.6412,187.0368,23.9295,32.8123,46.3418,162.8522,88.5518,192.7354,⋯,63.7012,99.1485,38.3684,140.3668,94.7103,70.4394,43.1736,33.6854,27.9604,16.2092
A1BG-AS1,36.3243,28.1315,23.2465,114.6008,10.4957,20.1900,17.2007,83.0906,65.6758,55.1265,⋯,30.3602,71.1797,26.0239,77.2415,61.4890,106.1623,47.8864,25.3482,17.9853,42.9450
A1CF,0.0000,0.0000,0.3310,0.0000,0.0000,0.0000,0.0000,0.9034,0.0000,0.0000,⋯,0.0000,0.0000,0.0000,1.9057,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A2M,5899.8279,9384.4401,3350.4207,1455.2316,3999.3792,3224.5797,1986.1593,1487.3247,10557.8325,4603.9013,⋯,10264.1373,7220.7664,7200.1144,1819.9428,6179.7358,5365.7972,8773.5936,10006.7877,1888.1746,4205.6055
A2M-AS1,118.4566,111.0026,71.3613,67.8607,52.1501,224.7582,182.0650,120.2838,53.6419,92.9112,⋯,146.6130,45.8648,64.4638,28.5565,100.6605,23.5625,84.2153,582.3492,59.8084,143.4910
A2ML1,7.5289,54.6875,5.6263,5.0883,3.3148,3.7421,6.2305,278.2486,4.7763,653.6058,⋯,1.7993,4.6124,0.4768,17.1510,15.6109,2.4461,98.3162,269.5768,4.5092,224.5290


In [5]:
data_mtx <- as.matrix(data)

## Load Danaher gene sets

In [6]:
Danaher_GeneSets <- read.csv('../../Data/3/DanaherImmuneCells_GeneSets.txt',
                             sep='\t',
                             header=T)

In [7]:
head(Danaher_GeneSets)

B.cells,CD45,CD8.T.cells,Cytotoxic.cells,DC,Exhausted.CD8,Macrophages,Mast.cells,Neutrophils,NK.CD56dim.cells,NK.cells,T.cells,Th1.cells,Treg
BLK,PTPRC,CD8A,CTSW,CCL13,CD244,CD163,MS4A2,CSF3R,IL21R,NCR1,CD3D,TBX21,FOXP3
CD19,,CD8B,GNLY,CD209,EOMES,CD68,TPSAB1,S100A12,KIR2DL3,XCL2,CD3E,,
MS4A1,,,GZMA,HSD11B1,LAG3,CD84,CPA3,CEACAM3,KIR3DL1,XCL1,CD3G,,
TNFRSF17,,,GZMB,,PTGER4,MS4A4A,HDC,FCAR,KIR3DL2,,CD6,,
FCRL2,,,GZMH,,,,TPSB2,FCGR3B,,,SH2D1A,,
KIAA0125,,,KLRB1,,,,,FPR1,,,TRAT1,,


In [8]:
Danaher_GeneSets <- convertRowsToList(t(Danaher_GeneSets))

## Run ssGSEA

In [9]:
dan_NES <- gsva(data_mtx,
                Danaher_GeneSets,
                method='ssgsea',
                min.sz=0,
                max.sz=1000,
                ssgsea.norm=T)

Warning message in .local(expr, gset.idx.list, ...):
“338 genes with constant expression values throuhgout the samples.”

Estimating ssGSEA scores for 14 gene sets.
  |                                                                      |   0%Using parallel with 4 cores
  |======================================================================| 100%


In [10]:
dan_NES <- data.frame("Term"=rownames(dan_NES),
                      dan_NES)

rownames(dan_NES) <- NULL

In [11]:
head(dan_NES)

Term,TCGA.04.1348.01,TCGA.04.1357.01,TCGA.04.1362.01,TCGA.04.1364.01,TCGA.04.1365.01,TCGA.04.1514.01,TCGA.04.1519.01,TCGA.09.0364.01,TCGA.09.0366.01,⋯,TCGA.61.2102.01,TCGA.61.2104.01,TCGA.61.2109.01,TCGA.61.2110.01,TCGA.61.2111.01,TCGA.61.2113.01,TCGA.OY.A56P.01,TCGA.OY.A56Q.01,TCGA.VG.A8LO.01,TCGA.WR.A838.01
B.cells,-0.213771868,-0.08940436,-0.343641848,-0.1893439,-0.23129310,-0.2873839,-0.2200140,-0.3480379,-0.33877468,⋯,-0.341824823,-0.1768512,-0.16194461,-0.11755471,-0.18095568,-0.23100966,-0.2574459,-0.3009135,-0.25201020,-0.2935001
CD45,0.237833263,0.42271477,0.006612357,-0.2482014,0.17573000,-0.2068874,-0.1653089,-0.2396847,0.31696995,⋯,0.010526873,0.2230216,0.08691282,-0.03057554,0.16647270,0.22720059,-0.1511320,-0.1361617,-0.08924037,-0.2201121
CD8.T.cells,0.038573990,0.23643827,-0.226109820,-0.2394391,-0.02170060,-0.2229383,-0.2539014,-0.2649932,-0.14371555,⋯,-0.209541112,-0.1471957,-0.20819160,-0.27474502,-0.07453824,-0.13273094,-0.2388512,-0.2670440,-0.22388449,-0.2727810
Cytotoxic.cells,-0.004140244,0.07281704,-0.209049699,-0.3349683,-0.03764848,-0.3387267,-0.2523178,-0.3662480,-0.16691094,⋯,-0.285398898,-0.1534489,-0.14751043,-0.23587506,-0.17941478,-0.17689138,-0.3563356,-0.3311566,-0.22302406,-0.3592876
DC,-0.136734293,-0.14755621,-0.232154900,-0.2631513,-0.18063379,-0.2249112,-0.1992590,-0.2168254,-0.23532651,⋯,0.001286832,-0.1382849,-0.19356849,-0.17074381,-0.20615901,-0.06518246,-0.3101320,-0.2962973,-0.32487374,-0.2708916
Exhausted.CD8,-0.062459317,0.03239466,-0.209199849,-0.3106540,-0.03781317,-0.2521386,-0.2443495,-0.2861368,-0.05061062,⋯,-0.227463099,-0.1089329,-0.19558487,-0.23015889,-0.17268619,-0.09331210,-0.1974754,-0.1833277,-0.23594305,-0.2300015


In [12]:
write.table(dan_NES,
            file='../../Data/3/DanaherImmuneCells_TCGAov_NES.txt',
            sep='\t',
            row.names=F,
            col.names=T)

# End script